# Métodos numéricos abordados en el proyecto para resolver la ecuación de Schrodinger unidimensional

Se presentará a grandes rasgos la idea de cada uno de los siguientes métodos, junto con la implementación de una función que permita aplicarlos directamente a una función previamente definida.

* Diferencias finitas centradas
* Runge-Kutta de orden 4
* Numerov
* Shooting
* Algoritmo QR

## Diferencias finitas centradas

Para aproximar la segunda derivada de una función $f(x)$ se utilizan diferencias centradas. A partir del desarrollo en series de Taylor de $f(x + h)$ y $f(x - h)$:

\begin{align*}
f(x + h) &= f(x) + h f'(x) + \frac{h^2}{2} f''(x) + \frac{h^3}{6} f^{(3)}(x) + \frac{h^4}{24} f^{(4)}(x) + \cdots \\
f(x - h) &= f(x) - h f'(x) + \frac{h^2}{2} f''(x) - \frac{h^3}{6} f^{(3)}(x) + \frac{h^4}{24} f^{(4)}(x) - \cdots
\end{align*}

Al sumar ambas expresiones y despejar $f''(x)$, se obtiene:

\begin{equation*}
f''(x) = \frac{f(x + h) - 2f(x) + f(x - h)}{h^2} - \frac{1}{12} h^2 f^{(4)}(x) + \cdots
\end{equation*}

La primera parte de la expresión es la **aproximación de segundo orden por diferencias centradas** para la segunda derivada. El segundo término representa el **error de truncamiento**, que es del orden $\mathcal{O}(h^2)$.

```python
def segundaDerivadaCentrada(f, x, h=1e-5):
    """
    f : Función a derivar, debe aceptar un float y devolver un float.
    x : Punto en el cual se evalúa la segunda derivada.
    h : Tamaño del paso para las diferencias finitas.
    """
    return (f(x + h) - 2 * f(x) + f(x - h)) / h**2

*[1] p. 406*

## Numerov

El método de Numerov es un esquema numérico de cuarto orden diseñado para resolver ecuaciones diferenciales ordinarias de segundo orden sin derivadas de primer orden, del tipo:
$$
\frac{d^2 y(x)}{dx^2} + k^2(x)\,y(x) = 0,
$$
como ocurre, por ejemplo, en la ecuación de Schrödinger independiente del tiempo en una dimensión:
$$
\frac{d^2 \psi(x)}{dx^2} + \frac{2m}{\hbar^2}(E - V(x))\,\psi(x) = 0.
$$

Este tipo de problemas son típicamente de **valores de frontera**, pues se requiere que la función de onda sea finita o tienda a cero en los extremos del dominio.

Se parte del desarrollo en serie de Taylor de $y(x \pm h)$ hasta el cuarto orden:
$$
y(x \pm h) = y(x) \pm h y'(x) + \frac{h^2}{2}y''(x) \pm \frac{h^3}{6}y'''(x) + \frac{h^4}{24}y^{(4)}(x) + \mathcal{O}(h^5).
$$

Al sumar estas dos expresiones:
$$
y(x + h) + y(x - h) = 2y(x) + h^2 y''(x) + \frac{h^4}{12}y^{(4)}(x) + \mathcal{O}(h^6).
$$

Despejando la segunda derivada:
$$
y''(x) = \frac{y(x + h) + y(x - h) - 2y(x)}{h^2} - \frac{h^2}{12} y^{(4)}(x).
$$

Ahora, como la ecuación original es $y''(x) = -k^2(x)\,y(x)$, se puede calcular la cuarta derivada usando que:
$$
y^{(4)}(x) \approx \frac{d^2}{dx^2}(-k^2(x)\,y(x)),
$$
y aproximar esta derivada en diferencias finitas.

Reorganizando, se llega a la fórmula de diferencias de Numerov:
$$
\left[1 + \frac{h^2}{12}k^2_{n+1}\right] y_{n+1} = 2\left[1 - \frac{5h^2}{12}k^2_n\right] y_n - \left[1 + \frac{h^2}{12}k^2_{n-1}\right] y_{n-1}.
$$

Despejando $y_{n+1}$:
$$
y_{n+1} = \frac{2\left(1 - \frac{5h^2}{12}k_n^2\right)y_n - \left(1 + \frac{h^2}{12}k_{n-1}^2\right)y_{n-1}}{1 + \frac{h^2}{12}k_{n+1}^2}.
$$

Esta fórmula permite propagar la solución numérica de manera eficiente y precisa desde dos condiciones iniciales, típicamente elegidas en una región donde se conoce el comportamiento de la función (por ejemplo, donde $\psi \sim 0$ fuera de la región de clase permitida).

```python
def numerov(V, x_min, x_max, h, E, psi_0=0.0, psi_1=1e-5):
    """
    - V: función del potencial V(x)
    - x_min, x_max: intervalo de posiciones
    - h: paso de integración
    - E: energía tentativa
    - psi_0, psi_1: condiciones iniciales (\psi(x0), \psi(x₀ + h))
    """
    m = 1.0     # masa reducida 
    hbar = 1.0  # Constante de Planck reducida 

    # Número de pasos espaciales a partir del intervalo y el tamaño del paso h
    N = int((x_max - x_min) / h)

    # Genera un array de N puntos espaciados uniformemente entre x_min y x_max
    x = np.linspace(x_min, x_max, N)

    # Calcula k^2(x) = (2m/\hbar^2)(E - V(x)) en cada punto del dominio
    k2 = 2 * m * (E - V(x)) / hbar**2

    psi = np.zeros(N)

    # Condiciones iniciales: \psi(x0) = psi_0 y \psi(x0 + h) = psi_1
    psi[0] = psi_0
    psi[1] = psi_1

    # Iteración del método de Numerov para obtener \psi(x) en todo el dominio
    for n in range(1, N - 1):
        psi[n + 1] = (
            (2 * (1 - (5 * h**2 * k2[n] / 12)) * psi[n] -  # Término central
            (1 + (h**2 * k2[n - 1] / 12)) * psi[n - 1]) / # Término anterior
            (1 + (h**2 * k2[n + 1] / 12))                  # Término siguiente (denominador)
        )
    return x, psi #x es un array de posiciones, psi es un array de eigenfunciones


[3], [4]

## Runge-Kutta de orden 4

El *método de Runge-Kutta de cuarto orden* es uno de los métodos más comunes y efectivos para resolver numéricamente ecuaciones diferenciales ordinarias (EDO) de primer orden. Proporciona un excelente balance entre precisión y simplicidad de implementación.

Es preciso hasta términos de orden $h^4$, con un error global del orden de $h^5$. Aunque su deducción completa mediante desarrollos de Taylor puede ser algebraicamente compleja, las ecuaciones resultantes son simples y prácticas para su implementación computacional.

El método calcula una nueva estimación $x(t + h)$ a partir de valores intermedios de la derivada $f(x, t)$ en el intervalo:

\begin{align*}
k_1 &= h f(x, t), \\
k_2 &= h f\left(x + \tfrac{1}{2}k_1, t + \tfrac{1}{2}h\right), \\
k_3 &= h f\left(x + \tfrac{1}{2}k_2, t + \tfrac{1}{2}h\right), \\
k_4 &= h f\left(x + k_3, t + h\right), \\
x(t + h) &= x(t) + \tfrac{1}{6}(k_1 + 2k_2 + 2k_3 + k_4).
\end{align*}

Cada $k_i$ representa una estimación de la pendiente en distintos puntos del paso, y su combinación ponderada proporciona una aproximación muy precisa del valor de la solución en el siguiente instante $t + h$.

```python
def rk4d(f, x0, a, b, h=0.01):
  '''
  f: función del sistema diferencial dx/dt = f(x, t)
  x0: condición inicial (valor de x en t = a)
  a, b: intervalo en t 
  h: paso de integración 
  '''
  xlista = []  # Lista para guardar posiciones
  tlista = np.arange(a, b + h, h)  # Lista de puntos en el tiempo desde a hasta b con paso h

  x = x0  # Establecemos la condicion inicial

  for i in tlista:
    xlista.append(x) 

    # Método de Runge-Kutta 4to orden:
    k1 = h * f(x, i)                            # Primera estimacion (como Euler)
    k2 = h * f(x + (k1 / 2), i + h / 2)         
    k3 = h * f(x + (k2 / 2), i + h / 2)         
    k4 = h * f(x + k3, i + h)                   # Evaluación al final del paso con k3

    # Combinacion ponderada para actualizar posicion
    x = x + (k1 + 2*k2 + 2*k3 + k4) / 6

  return tlista, xlista  # Regresamos la lista de tiempos y la solucion x(t)


*[1] pp. 331 - 338*

*[2] pp. 288 - 291*

## Shooting

El método de Shooting consiste en aplicar un método numérico estándar para resolver ecuaciones diferenciales (por ejemplo, el método de **Runge-Kutta de cuarto orden**) con el objetivo de calcular el valor de una función $f(v)$, la cual vincula las condiciones iniciales desconocidas con las condiciones de frontera finales. Posteriormente, se utiliza un método de búsqueda de raíces, (como el de la **secante**), para encontrar el valor de $v$ que hace que $f(v)$ cumpla con las condiciones de frontera especificadas.

Previamente ya hemos descrito y programado el método de **Runge-Kutta de orden cuatro**, por lo que a continuación completaremos la descripción del método de Shooting hablando sobre el **método de la secante.**

### Método de la secante

El *método de la secante* es una variante del método de Newton que permite encontrar raíces de funciones de la forma $f(x) = 0$, sin requerir el conocimiento explícito de la derivada $f'(x)$.

Cuando no se dispone de una fórmula analítica para la derivada, esta se puede aproximar mediante la pendiente entre dos puntos anteriores:

$$
f'(x_2) \approx \frac{f(x_2) - f(x_1)}{x_2 - x_1}
$$

Sustituyendo esta expresión en la fórmula del método de Newton se obtiene la fórmula iterativa de la secante:

$$
x_3 = x_2 - f(x_2) \cdot \frac{x_2 - x_1}{f(x_2) - f(x_1)}
$$

Esta expresión permite generar una nueva aproximación $x_3$ a la raíz de $f(x)$, utilizando únicamente evaluaciones de la función.

El método utiliza dos puntos anteriores $x_1$ y $x_2$, y construye  la *secante* entre los puntos $(x_1, f(x_1))$ y $(x_2, f(x_2))$. Esta línea recta sirve como una aproximación de la tangente en $x_2$, que es usada para predecir el siguiente valor $x_3$. A diferencia del método de Newton, que requiere derivadas exactas, aquí solo se emplean diferencias finitas.

```python
def secante(f, x0, x1, epsilon=1e-6):
  '''
  f: función de la que queremos encontrar una raíz
  x0, x1: primeras dos aproximaciones iniciales (se requieren dos)
  epsilon: tolerancia para determinar cuándo f(x) es suficientemente cercana a 0
  '''
  f0 = f(x0)  # Evaluamos la funcion en el primer punto
  f1 = f(x1)  # Evaluamos la funcion en el segundo punto

  while True:
    # Para evitar divisiones inseguras o errores de precision, usamos **(-1) en vez de "/"
    x2 = x1 - f1 * (x1 - x0) * (f1 - f0) ** (-1)

    f2 = f(x2)  # Evaluamos la funcion en la nueva aproximacion

    if abs(f2) < epsilon:
      break  # Criterio de convergencia
    
    # Actualizamos las variables para la siguiente iteracion
    f0 = f1
    f1 = f2
    x0 = x1
    x1 = x2

    return x2  # Regresa la mejor aproximacion de la raiz


*[1] pp. 273,274, 388 - 391*

*[2] pp. 674, 675*

# Algoritmo QR

El algoritmo QR permite calcular los **eigenvalores** y **eigenvectores** de una matriz simétrica real $A$, mediante una sucesión de descomposiciones ortogonales. En cada iteración, se descompone la matriz actual como $A = QR $, donde:

* $Q$ es una matriz ortogonal (sus columnas son ortonormales),
* $R$ es una matriz triangular superior.

Luego se construye una nueva matriz:

$$ A' = RQ $$

Repitiendo este proceso, la matriz converge hacia una forma diagonal:

$$
A_k \longrightarrow D \quad \text{(matriz diagonal)},
$$

cuyos elementos diagonales son los **eigenvalores** de la matriz original $A $, y las columnas del producto acumulado de matrices $ Q $:

$$
V = Q_1 Q_2 \cdots Q_k
$$

son los **eigenvectores** correspondientes.

Sea una matriz $A \in \mathbb{R}^{n \times n}$, simétrica y real. Tal que se construye la siguiente secuencia:

\begin{align*}
A_0 &= A = Q_1 R_1, \\
A_1 &= R_1 Q_1 = Q_1^T A Q_1, \\
A_2 &= Q_2^T A_1 Q_2 = Q_2^T Q_1^T A Q_1 Q_2, \\
&\vdots \\
A_k &= (Q_k^T \cdots Q_1^T) A (Q_1 \cdots Q_k).
\end{align*}

Esto es una **similitud ortogonal**, lo cual garantiza que todas las matrices $ A_k $ son similares entre sí y tienen los mismos autovalores que la matriz original $A$.

Finalmente, se define:

$$
V = Q_1 Q_2 \cdots Q_k, \quad D \approx A_k,
$$

de modo que:

$$
A \approx V D V^T,
$$

donde:
* $D$ es una matriz diagonal con los **eigenvalores**,
* Las columnas de $V$ son los **eigenvectores** ortonormales de $A$.

```python
def QRdescomp(A):
    '''
    Descompone una matriz cuadrada A en el producto QR usando Gram-Schmidt.
    A : ndarray (n x n) Matriz real cuadrada a descomponer.
    '''
    n = A.shape[0]  # Se obtiene el número de filas = columnas de A
    
    # Inicializamos Q y R como matrices de ceros
    Q = np.zeros_like(A, dtype=float)
    R = np.zeros((n, n), dtype=float)

    # Gram-Schmidt modificado
    for i in range(n):
        u = A[:, i].copy()  # Tomamos la i-ésima columna de A

        # Restamos las proyecciones sobre los vectores ortonormales anteriores
        for j in range(i):
            R[j, i] = np.dot(Q[:, j], A[:, i])  # Producto interno q_j · a_i
            u -= R[j, i] * Q[:, j]              # Restamos la componente en dirección de q_j

        R[i, i] = np.linalg.norm(u)  # Norma del vector ortogonal resultante
        Q[:, i] = u / R[i, i]        # Normalizamos para obtener q_i

    return Q, R  # Se devuelve la matriz ortogonal Q y la triangular superior R


```python
def QR(A, tol=1e-6):
    '''
    Calcula los eigenvalores y eigenvectores de una matriz simétrica real usando el algoritmo QR.
    A : ndarray (n x n). Matriz real y simétrica cuyas raíces (eigenvalores) se desean calcular.
    tol : Tolerancia para determinar la convergencia. El algoritmo termina cuando 
        todos los elementos fuera de la diagonal son menores que esta tolerancia.
    '''
    # Obtenemos el tamaño de la matriz cuadrada A
    n = A.shape[0]
    
    # Inicializamos Qtotal como la matriz identidad de tamaño n x n
    Qtotal = np.eye(n)
    
    while True:
        # Calculamos la descomposición QR de la matriz Ak
        Q, R = QRdescomp(A)
        
        # Obtenemos la nueva matriz Ak = R @ Q (producto de R por Q)
        Ak = R @ Q
        
        # Acumular el producto de Qs para construir la matriz de eigenvectores
        Qtotal = Qtotal @ Q
        
        # Construimos la matriz que contiene solo los elementos fuera de la diagonal
        offDiagonal = Ak - np.diag(np.diagonal(Ak))
        
        # Criterio de convergencia: los elementos fuera de la diagonal son suficientemente pequeños
        if np.all(np.abs(offDiagonal) < tol):
            break  

    # Los eigenvalores están en la diagonal de la matriz Ak final
    eigenvalores = np.diagonal(Ak)
    
    # Los eigenvectores se guardan en las columnas de Qtotal
    eigenvectores = Qtotal
    
    return eigenvalores, eigenvectores

*[1] pp. 241 - 248*

## Referencias

[1] Newman, M. E. J. (2013). *Computational physics*. CreateSpace Independent Publishing Platform.

[2] Burden, R. L., & Faires, J. D. (2010). *Numerical analysis* (9.ª ed.). Brooks/Cole, Cengage Learning.

[3] Caruso, F., Oguri, V., & Silveira, F. (2022). Applications of the Numerov method to simple quantum systems using Python. Revista Brasileira de Ensino de Física, 44, e20220098. https://doi.org/10.1590/1806-9126-RBEF-2022-0098

[4] Evans, M. (s.f.). Schroedinger Equation in Harmonic Potential [Código fuente]. Recuperado de https://mtdevans.com/js/sch.py.txt